# Final Projec - Gauging Brand Perception: How Luxury Brands Can Stay One Step Ahead.

Team members: Carley Wiley, Eldar Utiushev, Bek Tukhtasinov, Mira Mohan, Tammy Duong.

## 1. Data Cleaning and Processing

1. Load Raw Data

In [30]:
pip install langdetect
pip install scikit-learn

SyntaxError: invalid syntax (3863399607.py, line 1)

In [31]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from langdetect import detect

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/ulugsali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ulugsali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ulugsali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
# loading data
df = pd.read_csv('tiktok_comments.csv')

# we only need a text column in tiktok_comments dataset since it's the only
# thing in this dataset that can help us gauge the brand perception. 
df = df[['text']]

df.head()

,text
0,Maybe the simpsons are real and we are the car...
1,I think the designers do this on purpose
2,not me waiting for the model to do the back fl...
3,this was a collab they did with balenciaga yal...
4,the videos were created after 😂


2. Data Cleaning

In [49]:
import pandas as pd
import re
from langdetect import detect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

def clean_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Preprocess and clean data
df['cleaned_text'] = df['text'].apply(clean_text)
df['filtered_text'] = df['cleaned_text'].apply(remove_stopwords)
df['language'] = df['filtered_text'].apply(detect_language)
df = df[df['language'] == 'en']  # Keep only English language comments

# Handle missing values
df.dropna(subset=['filtered_text'], inplace=True)

df.head()

Cleaned data saved to 'cleaned_data.csv'.


In [52]:
df = df[['filtered_text']]

# Optionally, if you want to rename the column to something more general like 'text', you can do:
df.rename(columns={'filtered_text': 'text'}, inplace=True)

# Now 'df' contains only the column with the cleaned and filtered text
df.head()

KeyError: "None of [Index(['filtered_text'], dtype='object')] are in the [columns]"

In [53]:
df.to_csv('filtered_data.csv', index=False)

3. Splitting into Train, Test and Validation 


In [55]:
from sklearn.model_selection import train_test_split

# First, separate out a random 500 rows for validation
validation_df = df.sample(n=500, random_state=42)
remaining_df = df.drop(validation_df.index)  # Drop the validation rows from the original dataset

# Now, split the remaining data into training and testing sets
# We need to adjust the proportions since 500 rows are already taken by the validation set
remaining_train_percentage = 0.7 / (0.85)  # Adjusting for 70% of the original now being a different percentage of the remaining
train_df, test_df = train_test_split(remaining_df, test_size=(1 - remaining_train_percentage), random_state=42)

# Print the sizes of each dataset to verify
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(validation_df)}")  # Should be exactly 500
print(f"Testing set size: {len(test_df)}")

Training set size: 6772
Validation set size: 500
Testing set size: 1452


# 2. OpenAI API Labeling